# ConveyorGuard - 01. 전처리

**EDA 인사이트 기반 전처리 → 정규화 → 윈도우 생성 → joblib 저장**

- **Accelerator: GPU T4 x2**
- 입력:
  - `conveyorguard-aihub` (원본 이미지)
  - `conveyorguard-eda` (00_eda 결과: `eda_metadata.joblib`, `eda_summary.json`)
- 출력: `train_data.joblib`, `val_data.joblib`, `test_data.joblib`

---

### Pipeline
> `00_eda` → **`01_preprocess`** → `02/03`(병렬) → `04_dl_tuning` → `05_llm_comparison`

| 입력 데이터셋 | 출력 | Accelerator |
|--------------|------|-------------|
| `conveyorguard-aihub` + `conveyorguard-eda` | `train/val/test_data.joblib` | GPU T4 x2 |

### EDA → 전처리 연결
```
00_eda에서 발견한 인사이트:
  - 센서/라벨/외부환경 데이터 파싱 완료 → 재활용
  - 정규화 범위 → EDA 통계 기반 (하드코딩 ✕)
  - 윈도우 크기 → 심각(3) 구간 특성 기반
  - 클래스 가중치 → 불균형 비율 기반
  - 이미지만 원본에서 추가 로드 (하이브리드)
```

## 1. 환경 설정

In [1]:
import os
import sys
import json
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import joblib
import warnings
warnings.filterwarnings('ignore')

# 환경 정보
print(f"Python: {sys.version}")
try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA: {torch.version.cuda}")
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            mem = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
            print(f"GPU {i}: {props.name} ({mem / 1024**3:.1f} GB)")
except ImportError:
    print("PyTorch: not installed")

print(f"\n✅ 라이브러리 로드 완료")
_notebook_start = time.time()

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA: 12.6
GPU 0: Tesla T4 (14.6 GB)
GPU 1: Tesla T4 (14.6 GB)

✅ 라이브러리 로드 완료


In [2]:
# 경로 설정
DATA_DIR = Path('/kaggle/input/conveyorguard-aihub')  # 원본 (이미지용)
EDA_DIR = Path('/kaggle/input/conveyorguard-eda')      # EDA 결과
if not EDA_DIR.exists():
    EDA_DIR = Path('/kaggle/input/eda')  # fallback
OUTPUT_DIR = Path('/kaggle/working')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 설정
SENSOR_NAMES = ['NTC', 'PM1.0', 'PM2.5', 'PM10', 'CT1', 'CT2', 'CT3', 'CT4']
STATE_LABELS = {0: '정상', 1: '경미', 2: '중간', 3: '심각'}

print(f"📂 원본 데이터: {DATA_DIR} (exists={DATA_DIR.exists()})")
print(f"📂 EDA 결과: {EDA_DIR} (exists={EDA_DIR.exists()})")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 원본 데이터: /kaggle/input/conveyorguard-aihub (exists=True)
📂 EDA 결과: /kaggle/input/conveyorguard-eda (exists=True)
📁 출력 폴더: /kaggle/working


## 2. EDA 결과 로드 및 인사이트 확인

In [3]:
# EDA 결과 로드
print("📂 EDA 결과 로드 중...")
eda_meta = joblib.load(EDA_DIR / 'eda_metadata.joblib')
with open(EDA_DIR / 'eda_summary.json', 'r', encoding='utf-8') as f:
    eda_summary = json.load(f)

print(f"\n📊 EDA 인사이트 (전처리 근거):")
print(f"   전체 프레임: {eda_summary['data_info']['total_frames']:,}개")
print(f"   전체 세션: {eda_summary['data_info']['total_sessions']}개")

# 클래스 불균형
cd = eda_summary['class_distribution']
print(f"\n   클래스 분포:")
for cls in ['normal', 'minor', 'moderate', 'severe']:
    print(f"      {cls}: {cd[cls]:,}개")
print(f"   불균형 비율: {cd['imbalance_ratio']}:1")

# 센서 중요도
print(f"\n   센서-열화상태 상관도 (EDA):")
for name, corr in eda_summary['sensor_importance'].items():
    bar = '█' * int(corr * 20)
    print(f"      {name:>6}: {corr:.3f} {bar}")

# 외부환경
print(f"\n   외부환경 상관도: 매우 낮음 (<0.06) → 보조 피처(FiLM)로만 사용")

# 심각 클래스 특성
sa = eda_summary['severe_analysis']
print(f"\n   심각(3) 특성:")
print(f"      dominant 세션: {sa['dominant_sessions']}개 (전체 세션 중)")
print(f"      포함 세션: {sa['containing_sessions']}개")
print(f"      → 심각은 세션 전체가 아닌 '특정 구간'에서만 발생")
print(f"      → 윈도우 기반 분류 필수 (세션 단위 분류 시 심각 탐지 불가)")

# EDA 데이터 확인
print(f"\n📊 EDA 메타데이터:")
print(f"   all_sensors: {eda_meta['all_sensors'].shape}")
print(f"   all_externals: {eda_meta['all_externals'].shape}")
print(f"   all_states: {eda_meta['all_states'].shape}")
print(f"   all_sessions: {len(eda_meta['all_sessions'])}개")
print(f"   all_images: {len(eda_meta['all_images'])}개 (샘플만 — 전체 이미지는 원본에서 로드)")

print("\n✅ EDA 결과 로드 완료")

📂 EDA 결과 로드 중...

📊 EDA 인사이트 (전처리 근거):
   전체 프레임: 111,870개
   전체 세션: 341개

   클래스 분포:
      normal: 54,928개
      minor: 24,081개
      moderate: 24,191개
      severe: 8,670개
   불균형 비율: 6.34:1

   센서-열화상태 상관도 (EDA):
         NTC: 0.792 ███████████████
         CT2: 0.382 ███████
         CT1: 0.336 ██████
         CT4: 0.233 ████
        PM10: 0.214 ████
       PM2.5: 0.204 ████
       PM1.0: 0.201 ████
         CT3: 0.179 ███

   외부환경 상관도: 매우 낮음 (<0.06) → 보조 피처(FiLM)로만 사용

   심각(3) 특성:
      dominant 세션: 0개 (전체 세션 중)
      포함 세션: 289개
      → 심각은 세션 전체가 아닌 '특정 구간'에서만 발생
      → 윈도우 기반 분류 필수 (세션 단위 분류 시 심각 탐지 불가)

📊 EDA 메타데이터:
   all_sensors: (111870, 8)
   all_externals: (111870, 3)
   all_states: (111870,)
   all_sessions: 341개
   all_images: 198개 (샘플만 — 전체 이미지는 원본에서 로드)

✅ EDA 결과 로드 완료


## 3. EDA 기반 정규화 범위 설정

In [4]:
# EDA 데이터에서 실제 센서 범위 계산 (하드코딩 → 데이터 기반)
all_sensors_eda = eda_meta['all_sensors']  # (111870, 8)
all_ext_eda = eda_meta['all_externals']    # (111870, 3)

# 1~99 percentile (이상치 제거)
SENSOR_MIN = np.percentile(all_sensors_eda, 1, axis=0)
SENSOR_MAX = np.percentile(all_sensors_eda, 99, axis=0)

EXT_MIN = np.percentile(all_ext_eda, 1, axis=0)
EXT_MAX = np.percentile(all_ext_eda, 99, axis=0)

print("📊 EDA 기반 센서 정규화 범위 (1~99 percentile):")
for i, name in enumerate(SENSOR_NAMES):
    corr = eda_summary['sensor_importance'].get(name, 0)
    print(f"   {name:>6}: [{SENSOR_MIN[i]:.1f}, {SENSOR_MAX[i]:.1f}] (상관도: {corr})")

print(f"\n📊 EDA 기반 외부환경 정규화 범위:")
ext_names = ['Temperature', 'Humidity', 'Illuminance']
for i, name in enumerate(ext_names):
    print(f"   {name}: [{EXT_MIN[i]:.1f}, {EXT_MAX[i]:.1f}]")


def normalize_sensors(data: np.ndarray) -> np.ndarray:
    """센서 Min-Max 정규화 (EDA 기반 범위)"""
    result = (data - SENSOR_MIN) / (SENSOR_MAX - SENSOR_MIN + 1e-8)
    return np.clip(result, 0, 1)


def normalize_images(data: np.ndarray) -> np.ndarray:
    """이미지 Min-Max 정규화 (프레임별)"""
    img_min = data.min(axis=(-2, -1), keepdims=True)
    img_max = data.max(axis=(-2, -1), keepdims=True)
    return (data - img_min) / (img_max - img_min + 1e-8)


def normalize_external(data: np.ndarray) -> np.ndarray:
    """외부환경 정규화 (EDA 기반 범위)"""
    result = (data - EXT_MIN) / (EXT_MAX - EXT_MIN + 1e-8)
    return np.clip(result, 0, 1)


print("\n✅ EDA 기반 정규화 함수 정의 완료")

📊 EDA 기반 센서 정규화 범위 (1~99 percentile):
      NTC: [19.9, 60.8] (상관도: 0.792)
    PM1.0: [6.0, 40.0] (상관도: 0.201)
    PM2.5: [10.0, 48.0] (상관도: 0.204)
     PM10: [19.0, 88.0] (상관도: 0.214)
      CT1: [1.4, 112.8] (상관도: 0.336)
      CT2: [0.7, 203.2] (상관도: 0.382)
      CT3: [0.3, 129.9] (상관도: 0.179)
      CT4: [0.3, 108.2] (상관도: 0.233)

📊 EDA 기반 외부환경 정규화 범위:
   Temperature: [22.0, 26.0]
   Humidity: [27.0, 36.0]
   Illuminance: [151.0, 530.0]

✅ EDA 기반 정규화 함수 정의 완료


## 4. 윈도우 설정 (EDA 근거)

In [5]:
# EDA 근거:
#   - 심각(3)이 세션 전체가 아닌 특정 '구간'에서만 발생 (dominant 세션 0개)
#   - → 세션 단위 분류 시 심각 클래스 탐지 불가
#   - → 윈도우 기반 분류 필수
#   - WINDOW_SIZE=30 (약 30초): 충분한 시계열 패턴 포착
#   - STRIDE=10: 오버랩으로 심각 구간 경계도 포착
WINDOW_SIZE = 30
STRIDE = 10
N_WORKERS = mp.cpu_count()

print(f"⚙️ 윈도우: {WINDOW_SIZE}프레임, 스트라이드: {STRIDE}")
print(f"⚙️ 병렬 워커: {N_WORKERS}")
print(f"   (EDA 근거: 심각 클래스가 구간에서만 발생 → 윈도우 기반 분류 필수)")

⚙️ 윈도우: 30프레임, 스트라이드: 10
⚙️ 병렬 워커: 4
   (EDA 근거: 심각 클래스가 구간에서만 발생 → 윈도우 기반 분류 필수)


## 5. 원본 데이터 세션 매핑 (이미지 로드용)

In [6]:
# 원본 데이터에서 세션 폴더 쌍 찾기 (이미지 로드용)
def detect_source_label(parent_dir: Path):
    folders = sorted(parent_dir.iterdir())
    source, label = None, None
    for f in folders:
        name = f.name
        if '원천' in name or 'source' in name.lower() or '01.' in name:
            source = f
        elif '라벨' in name or 'label' in name.lower() or '02.' in name:
            label = f
    if source is None or label is None:
        source, label = folders[0], folders[1]
    return source, label

def find_zip_pairs(source_dir: Path, label_dir: Path):
    def get_key(filename):
        stem = filename.stem
        if stem[:3] in ['TS_', 'TL_', 'VS_', 'VL_']:
            return stem[3:]
        return stem
    source_zips = {get_key(z): z for z in source_dir.iterdir() if z.name.endswith('.zip') or z.is_dir()}
    label_zips = {get_key(z): z for z in label_dir.iterdir() if z.name.endswith('.zip') or z.is_dir()}
    return [(source_zips[k], label_zips[k]) for k in source_zips if k in label_zips]

train_source, train_label = detect_source_label(DATA_DIR / 'Training')
val_source, val_label = detect_source_label(DATA_DIR / 'Validation')

train_pairs = find_zip_pairs(train_source, train_label)
val_pairs = find_zip_pairs(val_source, val_label)

print(f"📊 Training 세션: {len(train_pairs)}개")
print(f"📊 Validation 세션: {len(val_pairs)}개")
print(f"   (센서/라벨/외부환경은 EDA에서 로드, 이미지만 원본에서 파싱)")

📊 Training 세션: 303개
📊 Validation 세션: 38개
   (센서/라벨/외부환경은 EDA에서 로드, 이미지만 원본에서 파싱)


## 6. 세션 데이터 로드 (하이브리드: EDA + 원본 이미지)

In [7]:
def load_session_with_images(source_dir: Path, label_dir: Path) -> Optional[Dict]:
    """원본에서 전체 데이터 로드 (이미지 포함)
    
    하이브리드 접근:
    - 센서/라벨/외부환경: 원본에서 파싱 (EDA와 동일 로직)
    - 이미지: 원본 BIN에서 로드 + 다운샘플링 (60x80)
    - EDA에는 이미지 전체가 저장되지 않았으므로 원본 필요
    """
    try:
        csv_files = sorted(source_dir.glob('*.csv'))
        bin_files = sorted(source_dir.glob('*.bin'))
        json_files = sorted(label_dir.glob('*.json'))
        
        n_frames = min(len(csv_files), len(bin_files), len(json_files))
        if n_frames < WINDOW_SIZE:
            return None
        
        sensors, images, externals, states = [], [], [], []
        
        for i in range(n_frames):
            # CSV 센서
            df = pd.read_csv(csv_files[i])
            if len(df.columns) >= 8:
                sensors.append(df.iloc[0, :8].values.astype(np.float32))
            else:
                continue
            
            # BIN 이미지 (다운샘플링 120x160 → 60x80)
            img = np.load(bin_files[i])
            if img.shape == (120, 160):
                images.append(img[::2, ::2].astype(np.float32))
            else:
                sensors.pop()  # 이미지 실패 시 센서도 제거
                continue
            
            # JSON 라벨 + 외부환경
            with open(json_files[i], 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            state = 0
            annotations = data.get('annotations', [])
            if isinstance(annotations, list) and len(annotations) > 0:
                tagging = annotations[0].get('tagging', [])
                if isinstance(tagging, list) and len(tagging) > 0:
                    state_val = tagging[0].get('state', '0')
                    state = int(state_val) if state_val else 0
            states.append(state)
            
            ext_list = data.get('external_data', [])
            ext = ext_list[0] if ext_list else {}
            def get_val(key):
                val = ext.get(key, [{}])
                return float(val[0].get('value', 0)) if isinstance(val, list) and val else 0.0
            externals.append([get_val('ex_temperature'), get_val('ex_humidity'), get_val('ex_illuminance')])
        
        if len(sensors) < WINDOW_SIZE:
            return None
        
        return {
            'sensors': np.array(sensors, dtype=np.float32),
            'images': np.array(images, dtype=np.float32),
            'externals': np.array(externals, dtype=np.float32),
            'states': np.array(states, dtype=np.int64)
        }
    except:
        return None

print("✅ 하이브리드 로드 함수 정의 완료")

✅ 하이브리드 로드 함수 정의 완료


## 7. 윈도우 생성

In [8]:
def create_windows(session_data: Dict, session_id: str = '') -> List[Dict]:
    """슬라이딩 윈도우로 샘플 생성
    
    EDA 근거: 심각(3)이 구간에서만 발생하므로
    윈도우 내 다수결(majority vote)로 라벨 결정
    """
    sensors = session_data['sensors']
    images = session_data['images']
    externals = session_data['externals']
    states = session_data['states']
    
    windows = []
    for start in range(0, len(sensors) - WINDOW_SIZE + 1, STRIDE):
        end = start + WINDOW_SIZE
        label = int(np.bincount(states[start:end]).argmax())
        windows.append({
            'sensors': sensors[start:end],
            'images': images[start:end],
            'externals': externals[start:end],
            'label': label,
            'session_id': session_id
        })
    return windows

print("✅ 윈도우 생성 함수 정의 완료")

✅ 윈도우 생성 함수 정의 완료


## 8. 전처리 실행

In [9]:
def process_one_session(pair):
    source_dir, label_dir = pair
    session_id = source_dir.name
    session_data = load_session_with_images(source_dir, label_dir)
    if session_data is None:
        return []
    return create_windows(session_data, session_id=session_id)


def process_split(pairs, split_name):
    print(f"\n🔄 {split_name} 전처리 ({N_WORKERS} workers)...")
    
    all_windows = []
    with ProcessPoolExecutor(max_workers=N_WORKERS) as executor:
        results = list(tqdm(executor.map(process_one_session, pairs),
                           total=len(pairs), desc=split_name))
    for windows in results:
        all_windows.extend(windows)
    
    if not all_windows:
        print(f"  ❌ {split_name}: 데이터 없음!")
        return None
    
    sensors = np.array([w['sensors'] for w in all_windows], dtype=np.float32)
    images = np.array([w['images'] for w in all_windows], dtype=np.float32)
    externals = np.array([w['externals'] for w in all_windows], dtype=np.float32)
    labels = np.array([w['label'] for w in all_windows], dtype=np.int64)
    session_ids = np.array([w['session_id'] for w in all_windows])
    
    # EDA 기반 정규화
    print(f"  정규화 중 (EDA 기반 범위)...")
    sensors = normalize_sensors(sensors)
    images = normalize_images(images)
    externals = normalize_external(externals)
    
    print(f"  ✅ {split_name}: {len(labels)}개 샘플, {len(np.unique(session_ids))}개 세션")
    unique, counts = np.unique(labels, return_counts=True)
    print(f"     라벨 분포: {dict(zip(unique, counts))}")
    
    return {'sensors': sensors, 'images': images, 'externals': externals,
            'labels': labels, 'session_ids': session_ids}


train_data = process_split(train_pairs, 'Training')
val_data = process_split(val_pairs, 'Validation')


🔄 Training 전처리 (4 workers)...


Training:   0%|          | 0/303 [00:00<?, ?it/s]

  정규화 중 (EDA 기반 범위)...
  ✅ Training: 9313개 샘플, 303개 세션
     라벨 분포: {np.int64(0): np.int64(4584), np.int64(1): np.int64(2133), np.int64(2): np.int64(2085), np.int64(3): np.int64(511)}

🔄 Validation 전처리 (4 workers)...


Validation:   0%|          | 0/38 [00:00<?, ?it/s]

  정규화 중 (EDA 기반 범위)...
  ✅ Validation: 1160개 샘플, 38개 세션
     라벨 분포: {np.int64(0): np.int64(522), np.int64(1): np.int64(287), np.int64(2): np.int64(285), np.int64(3): np.int64(66)}


In [10]:
# EDA 결과와 교차 검증
total_parsed = len(train_data['labels']) + len(val_data['labels'])
# 윈도우 수 ≠ 프레임 수이므로 세션 수로 검증
parsed_sessions = len(np.unique(np.concatenate([train_data['session_ids'], val_data['session_ids']])))
eda_sessions = eda_summary['data_info']['total_sessions']

print(f"\n🔍 EDA 교차 검증:")
print(f"   EDA 세션: {eda_sessions}개 vs 파싱 세션: {parsed_sessions}개")
print(f"   총 윈도우: {total_parsed}개")

if abs(eda_sessions - parsed_sessions) <= 5:
    print("   ✅ 세션 수 일치 (허용 범위 내)")
else:
    print(f"   ⚠️ 세션 수 차이: {abs(eda_sessions - parsed_sessions)}개")


🔍 EDA 교차 검증:
   EDA 세션: 341개 vs 파싱 세션: 341개
   총 윈도우: 10473개
   ✅ 세션 수 일치 (허용 범위 내)


## 9. 세션 단위 Split + 클래스 가중치 (EDA 기반)

In [11]:
from sklearn.model_selection import GroupShuffleSplit

print("🔀 세션 단위 Train/Val/Test 분리 (70/15/15)...")
print("   ⚠️ 같은 세션의 윈도우가 train/test에 겹치지 않도록 세션 그룹 기반 split")

# 전체 합치기
all_sensors = np.concatenate([train_data['sensors'], val_data['sensors']])
all_images = np.concatenate([train_data['images'], val_data['images']])
all_externals = np.concatenate([train_data['externals'], val_data['externals']])
all_labels = np.concatenate([train_data['labels'], val_data['labels']])
all_session_ids = np.concatenate([train_data['session_ids'], val_data['session_ids']])

print(f"   전체: {len(all_labels)}개 윈도우, {len(np.unique(all_session_ids))}개 세션")

# 세션 단위 split
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, temp_idx = next(gss1.split(all_labels, all_labels, groups=all_session_ids))

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
val_rel, test_rel = next(gss2.split(all_labels[temp_idx], all_labels[temp_idx],
                                     groups=all_session_ids[temp_idx]))
val_idx, test_idx = temp_idx[val_rel], temp_idx[test_rel]

# 누출 검증
train_sess = set(all_session_ids[train_idx])
val_sess = set(all_session_ids[val_idx])
test_sess = set(all_session_ids[test_idx])
assert len(train_sess & val_sess) == 0, "Train-Val 세션 겹침!"
assert len(train_sess & test_sess) == 0, "Train-Test 세션 겹침!"
assert len(val_sess & test_sess) == 0, "Val-Test 세션 겹침!"
print("   ✅ 세션 누출 검증 통과")

# EDA 기반 클래스 가중치 계산
class_counts = np.bincount(all_labels[train_idx])
class_weights = 1.0 / class_counts
class_weights = (class_weights / class_weights.sum() * len(class_counts)).astype(np.float32)

print(f"\n   📊 클래스 가중치 (EDA 불균형 {eda_summary['class_distribution']['imbalance_ratio']}:1 대응):")
for i, w in enumerate(class_weights):
    print(f"      {i}({STATE_LABELS[i]}): {w:.3f}")

# 분리
train_final = {'sensors': all_sensors[train_idx], 'images': all_images[train_idx],
               'external': all_externals[train_idx], 'labels': all_labels[train_idx],
               'class_weights': class_weights}
val_final = {'sensors': all_sensors[val_idx], 'images': all_images[val_idx],
             'external': all_externals[val_idx], 'labels': all_labels[val_idx]}
test_final = {'sensors': all_sensors[test_idx], 'images': all_images[test_idx],
              'external': all_externals[test_idx], 'labels': all_labels[test_idx]}

for name, idx, sess in [('Train', train_idx, train_sess), ('Val', val_idx, val_sess), ('Test', test_idx, test_sess)]:
    u, c = np.unique(all_labels[idx], return_counts=True)
    print(f"   {name}: {len(idx)} ({len(idx)/len(all_labels)*100:.1f}%) — {len(sess)}개 세션 — {dict(zip(u,c))}")

🔀 세션 단위 Train/Val/Test 분리 (70/15/15)...
   ⚠️ 같은 세션의 윈도우가 train/test에 겹치지 않도록 세션 그룹 기반 split
   전체: 10473개 윈도우, 341개 세션
   ✅ 세션 누출 검증 통과

   📊 클래스 가중치 (EDA 불균형 6.34:1 대응):
      0(정상): 0.281
      1(경미): 0.598
      2(중간): 0.613
      3(심각): 2.509
   Train: 7311 (69.8%) — 238개 세션 — {np.int64(0): np.int64(3586), np.int64(1): np.int64(1682), np.int64(2): np.int64(1642), np.int64(3): np.int64(401)}
   Val: 1554 (14.8%) — 51개 세션 — {np.int64(0): np.int64(732), np.int64(1): np.int64(367), np.int64(2): np.int64(367), np.int64(3): np.int64(88)}
   Test: 1608 (15.4%) — 52개 세션 — {np.int64(0): np.int64(788), np.int64(1): np.int64(371), np.int64(2): np.int64(361), np.int64(3): np.int64(88)}


## 10. joblib 저장

In [12]:
print("💾 joblib 저장 중...")
joblib.dump(train_final, OUTPUT_DIR / 'train_data.joblib', compress=3)
joblib.dump(val_final, OUTPUT_DIR / 'val_data.joblib', compress=3)
joblib.dump(test_final, OUTPUT_DIR / 'test_data.joblib', compress=3)

for name in ['train', 'val', 'test']:
    size = os.path.getsize(OUTPUT_DIR / f'{name}_data.joblib') / (1024**2)
    print(f"   {name}_data.joblib: {size:.1f} MB")

print("\n✅ 저장 완료 (세션 단위 split, 데이터 누출 방지, EDA 기반 정규화)")

💾 joblib 저장 중...
   train_data.joblib: 2751.9 MB
   val_data.joblib: 584.7 MB
   test_data.joblib: 600.4 MB

✅ 저장 완료 (세션 단위 split, 데이터 누출 방지, EDA 기반 정규화)


## 11. 데이터 검증 및 다운로드

In [4]:
import os

os.environ['KAGGLE_USERNAME'] = 'kukass'
os.environ['KAGGLE_KEY'] = 'KGAT_c973fff8eb3e1ccb19f3e9d683eb17dc'

# 데이터셋 업로드
!kaggle datasets create -p /kaggle/working/dataset_upload/ --dir-mode zip

Starting upload for file val_data.joblib
100%|█████████████████████████████████████████| 585M/585M [00:05<00:00, 103MB/s]
Upload successful: val_data.joblib (585MB)
Starting upload for file test_data.joblib
100%|█████████████████████████████████████████| 600M/600M [00:05<00:00, 114MB/s]
Upload successful: test_data.joblib (600MB)
Starting upload for file train_data.joblib
100%|███████████████████████████████████████| 2.69G/2.69G [00:22<00:00, 128MB/s]
Upload successful: train_data.joblib (3GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/kukass/conveyorguard-preprocess
